<a href="https://colab.research.google.com/github/00hamza00/Data-Science-and-Decision-Making/blob/master/Lab5/my_recommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import itertools

from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import accuracy_score as acc
from sklearn.metrics import make_scorer, confusion_matrix
from sklearn.dummy import DummyRegressor, DummyClassifier
from sklearn.preprocessing import LabelEncoder

In [0]:
df_jester1 = pd.read_csv ('https://raw.githubusercontent.com/albanda/CE888/master/lab5-recommender/jester-data-1.csv',delimiter=',',header=None)

df_jester1.head()

In [0]:
# making the dataset into an array

arr = df_jester1.values
print(arr.shape)

In [0]:
rated = np.where(arr!=99)
print(len(rated[0]), rated[1].shape)

In [0]:
def replace(orig, percentage=0.1):
  """
  Replaces 'percentage'% of the original values in 'orig' with 99's
  :param orig: original data array
  :param percentage: percentage of values to replace (0<percentage<1)
  """
  new_data = orig.copy()
  rated = np.where(arr!=99)
  n_rated = len(rated[0])
  idx = np.random.choice(n_rated, size=int(percentage*n_rated), replace=False)
  new_data[rated[0][idx], rated[1][idx]] = 99
  return new_data, (rated[0][idx], rated[1][idx])

In [0]:
new_arr, idx = replace(arr, 0.1)
arr[idx[0][0], idx[1][0]]

In [0]:

new_arr[idx[0][0], idx[1][0]]

In [0]:
n_latent_factors = 2
jokes_ratings = new_arr
# Initialise as random values
latent_user_preferences = np.random.random((jokes_ratings.shape[0], n_latent_factors))
latent_item_features = np.random.random((jokes_ratings.shape[1], n_latent_factors))

In [0]:
latent_item_features

In [0]:
latent_user_preferences

In [0]:
def predict_rating(user_id, item_id):
    """ Predict a rating given a user_id and an item_id.
    """
    user_preference = latent_user_preferences[user_id]
    item_preference = latent_item_features[item_id]
    return user_preference.dot(item_preference)


def train(user_id, item_id, rating, alpha=0.0001):
    
    #print item_id
    prediction_rating = predict_rating(user_id, item_id)
    err =  prediction_rating - rating
    #print err
    user_pref_values = latent_user_preferences[user_id][:]
    latent_user_preferences[user_id] -= alpha * err * latent_item_features[item_id]
    latent_item_features[item_id] -= alpha * err * user_pref_values
    return err
    

def sgd(iterations=50):
    """ Iterate over all users and all items and train for 
        a certain number of iterations
    """
    for iteration in range(iterations):
        error = []
        for user_id in range(latent_user_preferences.shape[0]):
            for item_id in range(latent_item_features.shape[0]):
                rating = jokes_ratings[user_id][item_id]
                if  rating!=99:
                    err = train(user_id, item_id, rating)
                    error.append(err)
        mse = (np.array(error) ** 2).mean()   
        if (iteration % 1) == 0:
            print(mse)

In [29]:
sgd()

24.386293718250244
24.31470964339305
24.202040446238804
24.03882653115196
23.805045689084956
23.477347298428857
23.035336245703636
22.47185346402506
21.805424978904377
21.086220939899516
20.385263986445505
19.76820792851066
19.271509066816044
18.897766958808166
18.6280053985082
18.436864148107137
18.30169297863707
18.20535452913323
18.13582245556301
18.084901478563268
18.047040560960475
18.018461257512886
17.99656703465099
17.979553133840483
17.966149792748677
17.95545240225378
17.946808286621017
17.939740566559795
17.933896482563682
17.92901197376978
17.924887143570484
17.921369076576674
17.918339664174447
17.915706875894877
17.913398426436842
17.911357126825827
17.90953743320148
17.90790285724035
17.90642400367911
17.90507706939495
17.903842685863513
17.90270501966617
17.901651068751015
17.900670108478295
17.89975325318294
17.898893107465206
17.898083487629638
17.897319198284155
17.89659585254133
17.89590972684747


In [30]:
predictions = latent_user_preferences.dot(latent_item_features.T)
predictions

array([[ 77.866611  ,  -2.65839418,  -3.79729249, ...,  -3.44107744,
         -4.39695762,  -3.48692857],
       [ 94.61371979,   3.02045623,   2.61064769, ...,   3.06076043,
          2.12631331,   3.64014208],
       [108.93410568,   6.3899169 ,   6.37190711, ...,   6.89820072,
          5.92809204,   7.86118002],
       ...,
       [ 39.08170772,  -0.55960729,  -1.01051157, ...,  -0.82958297,
         -1.28120975,  -0.77388675],
       [ 38.30262231,  -0.2453037 ,  -0.63997945, ...,  -0.46181932,
         -0.89343286,  -0.37643197],
       [ 72.76830747,   3.16817319,   2.98468015, ...,   3.33320313,
          2.64520499,   3.86467884]])

In [0]:
values = [zip(jokes_ratings[i], predictions[i]) for i in range(predictions.shape[0])]
comparison_data = pd.DataFrame(values)
data= pd.DataFrame(data=new_arr,    # values
              index=None,    # 1st column as index
              columns=None)
comparison_data.columns = data.columns

In [35]:
comparison_data
# For each data point, the number on the left is the original value from the dataset, the number on the right is the prediction

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100
0,"(74.0, 77.8666109998722)","(99.0, -2.658394184533312)","(8.79, -3.797292491511992)","(-9.66, -3.859038790260496)","(-8.16, -6.034258305771455)","(-7.52, -2.616870108868417)","(-8.5, -3.0683462831112496)","(-9.85, -3.2352781973225784)","(4.17, -2.344517094980643)","(-8.98, -5.1368942957045265)","(-4.76, -2.468634486870938)","(-8.5, -2.5196200822945554)","(-6.75, -3.3172640999108958)","(-7.18, -3.8778820461504107)","(8.45, -3.352524881486426)","(-7.18, -3.3758848294559316)","(-7.52, -4.97275759007091)","(-7.43, -3.34959254881731)","(-9.81, -3.1809517738223074)","(-9.85, -3.027956081692281)","(-9.85, -4.455876030996345)","(-9.37, -2.121872378028303)","(1.5, -3.7607019473998555)","(-4.37, -4.2023593857560275)","(-9.81, -5.94842394884286)","(-8.5, -4.207893522088594)","(1.12, -3.3740019357240154)","(7.82, -0.17795544002477848)","(2.86, -2.7967421048171994)","(9.13, -0.21603888705749894)","(-7.43, -5.492472304242113)","(2.14, -1.8155168305795302)","(99.0, -0.24086979693862143)","(-9.08, -5.9093548151111035)","(7.82, -3.553111051217462)","(5.05, -0.5709661645782137)","(4.95, 0.18519631725341199)","(-9.17, -5.233287545660497)","(-8.4, -3.014212330394393)","(-8.4, -3.5967108468637354)",...,"(99.0, -1.7808210955974058)","(3.59, -0.8204682306174799)","(-6.84, -4.349882102438381)","(99.0, -4.861035837434797)","(2.82, -1.698247361247385)","(-1.36, -1.3192806828480432)","(-9.08, -5.202828977606145)","(8.3, -1.1184555798132523)","(5.68, -1.3589143759678972)","(-4.81, -4.033574400060167)","(99.0, -4.7592982107520605)","(99.0, -1.638264401462639)","(99.0, -3.8440016403948487)","(99.0, -5.717283453129075)","(99.0, -4.327530347927118)","(99.0, -1.6787610571024185)","(99.0, -3.8715079296310813)","(-9.42, -3.1308333216063815)","(99.0, -4.67592214077279)","(99.0, -3.974036075629949)","(99.0, -2.6112098599472526)","(-7.72, -3.616790555127138)","(99.0, -2.311356666231788)","(99.0, -3.8692908784788185)","(99.0, -4.038109683694972)","(99.0, -3.5254728988984683)","(99.0, -2.284217422060539)","(99.0, -1.967685156607538)","(99.0, -0.23755042370407678)","(99.0, -3.974079112590207)","(2.82, -1.952975233541754)","(99.0, -3.2450126226423985)","(99.0, -1.6948139020610358)","(99.0, -3.072061226334219)","(99.0, -2.941981091764927)","(99.0, -2.7636395548466273)","(-5.63, -2.3074137457766764)","(99.0, -3.4410774363610783)","(99.0, -4.396957620159022)","(99.0, -3.486928566000632)"
1,"(100.0, 94.61371979332931)","(4.08, 3.020456225056919)","(-0.29, 2.6106476896231077)","(6.36, 2.6751035268743157)","(4.37, 1.0579976160386204)","(-2.38, 2.085841671216235)","(-9.66, 4.166098893214009)","(-0.73, 1.0419434201130042)","(-5.34, 0.2536587071842017)","(8.88, 1.798038962139098)","(9.22, 3.4070141501417233)","(99.0, 4.337572252146393)","(8.64, 4.228656311225623)","(4.42, -1.004364434899677)","(99.0, 4.204963256647414)","(4.56, -1.2502561662865626)","(-0.97, -2.7250593109622505)","(4.66, -0.06345661605051452)","(-0.68, 0.498506043605993)","(3.3, 1.876488431609478)","(-1.21, 0.6818578932997821)","(0.87, 4.869711202401503)","(8.64, 3.493011230776481)","(8.35, 2.5470296285709955)","(9.17, 0.5213954049586361)","(0.05, 2.960358042710452)","(7.57, 3.984582224683078)","(4.71, 5.624500877228619)","(0.87, 4.035745330833089)","(-0.39, 5.324897956867822)","(6.99, 2.1231567503510007)","(6.5, 4.8294515634550725)","(-0.92, 5.601258843852774)","(99.0, 1.0280381869219453)","(9.03, 3.5004989219064893)","(-1.8, 5.564167425526462)","(0.73, 5.638393682821405)","(7.09, 0.5873663090628481)","(99.0, 3.641066124158076)","(-0.87, 3.7658917923751005)",...,"(-6.7, 5.124534834803248)","(-3.35, 5.563225228836058)","(-9.03, 2.583586507791012)","(4.47, 1.4639306774046776)","(4.08, 4.909807613838858)","(-3.83, 5.078829201448275)","(8.74, 1.4133754733149533)","(1.12, 5.071845

In [36]:
df_jester1


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100
0,74,-7.82,8.79,-9.66,-8.16,-7.52,-8.50,-9.85,4.17,-8.98,-4.76,-8.50,-6.75,-7.18,8.45,-7.18,-7.52,-7.43,-9.81,-9.85,-9.85,-9.37,1.50,-4.37,-9.81,-8.50,1.12,7.82,2.86,9.13,-7.43,2.14,-4.08,-9.08,7.82,5.05,4.95,-9.17,-8.40,-8.40,...,8.59,3.59,-6.84,-9.03,2.82,-1.36,-9.08,8.30,5.68,-4.81,99.00,99.00,99.00,99.00,99.00,99.00,99.00,-9.42,99.00,99.00,99.00,-7.72,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,2.82,99.00,99.00,99.00,99.00,99.00,-5.63,99.00,99.00,99.00
1,100,4.08,-0.29,6.36,4.37,-2.38,-9.66,-0.73,-5.34,8.88,9.22,6.75,8.64,4.42,7.43,4.56,-0.97,4.66,-0.68,3.30,-1.21,0.87,8.64,8.35,9.17,0.05,7.57,4.71,0.87,-0.39,6.99,6.50,-0.92,7.14,9.03,-1.80,0.73,7.09,3.40,-0.87,...,-6.70,-3.35,-9.03,4.47,4.08,-3.83,8.74,1.12,0.78,7.52,-5.00,2.77,8.30,7.77,7.33,6.21,7.72,8.98,8.64,8.20,3.93,4.85,4.85,6.07,8.98,4.51,-0.05,3.69,4.56,0.58,2.82,-4.95,-0.29,7.86,-0.19,-2.14,3.06,0.34,-4.32,1.07
2,49,99.00,99.00,99.00,99.00,9.03,9.27,9.03,9.27,99.00,99.00,7.33,7.57,9.37,6.17,-6.36,-6.89,-7.86,9.03,9.03,9.03,7.28,99.00,8.25,99.00,99.00,7.48,7.28,7.28,8.93,99.00,6.17,7.28,99.00,99.00,8.98,7.33,99.00,6.17,9.08,...,6.46,7.28,99.00,99.00,7.04,7.28,99.00,7.28,8.25,99.00,99.00,99.00,99.00,99.00,99.00,8.93,99.00,99.00,99.00,9.08,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,9.03,99.00,99.00,99.00,9.08,99.00,99.00,99.00,99.00,99.00,99.00
3,48,99.00,8.35,99.00,99.00,1.80,8.16,-2.82,6.21,99.00,1.84,7.33,6.60,6.31,8.11,-7.23,-6.65,1.17,-6.60,-3.64,-2.09,5.34,99.00,99.00,99.00,99.00,2.91,3.93,6.75,6.60,99.00,6.65,-6.12,99.00,7.57,6.21,6.65,99.00,-8.30,7.18,...,0.00,-3.69,99.00,99.00,7.82,0.24,99.00,7.28,-2.33,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,0.63,99.00,99.00,-2.33,99.00,99.00,99.00,99.00,99.00,0.53,99.00,99.00,99.00,99.00,99.00,99.00
4,91,8.50,4.61,-4.17,-5.39,1.36,1.60,7.04,4.61,-0.44,5.73,8.25,6.84,-3.93,7.23,-2.33,-9.66,2.72,-1.36,2.57,4.51,8.20,6.12,8.30,-1.26,7.77,1.89,-1.17,5.68,8.45,4.61,8.06,-9.47,7.28,5.68,2.48,3.20,-1.26,6.80,4.51,...,7.38,6.17,4.71,-2.28,7.38,4.56,7.14,4.22,3.01,3.83,99.00,99.00,99.00,99.00,99.00,99.00,4.13,99.00,99.00,99.00,5.24,5.92,0.87,7.28,3.93,-0.63,6.31,4.71,2.82,2.96,5.19,5.58,4.27,5.19,5.73,1.55,3.11,6.55,1.80,1.60
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24978,100,0.44,7.43,9.08,2.33,3.20,6.75,-8.79,-0.53,-8.74,7.23,-0.53,5.63,-7.14,-4.08,-3.50,-8.20,-3.98,-9.22,-0.15,-6.46,5.63,-0.92,-2.91,-4.17,2.82,3.40,8.64,6.84,6.80,-0.87,7.38,-3.50,8.88,7.43,5.39,2.23,-0.68,3.40,-0.58,...,8.59,3.45,0.87,9.27,-4.66,5.73,-0.49,8.35,1.94,5.00,-9.66,8.98,8.98,-9.81,9.13,9.08,9.08,3.98,0.73,9.03,8.98,9.22,8.93,9.13,9.27,-1.99,-9.95,-9.90,9.13,8.83,8.83,-1.21,9.22,-6.70,8.45,9.03,6.55,8.69,8.79,7.43
24979,91,9.13,-8.16,8.59,9.08,0.87,-8.93,-3.50,5.78,-8.11,4.90,8.88,-8.69,-7.48,-8.83,-1.75,6.60,3.54,1.50,7.67,-0.44,9.22,8.74,9.03,9.08,8.93,3.74,3.20,-9.17,-8.98,8.79,-7.67,-3.06,9.13,8.40,-0.63,-7.18,0.58,8.88,9.27,...,2.77,8.11,-7.96,8.93,-0.87,-5.87,8.88,-1.12,-8.74,8.74,99.00,99.00,99.00,99.00,99.00,4.90,99.00,99.00,99.00,99.00,-0.29,0.92,-0.78,0.15,-0.10,0.00,-0.19,-0.87,-1.36,-0.58,-1.17,-5.73,-1.46,0.24,9.22,-8.20,-7.23,-8.59,9.13,8.45
24980,39,99.00,99.00,99.00,99.00,-7.77,99.00,6.70,-6.75,99.00,99.00,99.00,99.00,-6.46,-1.65,-6.80,-6.41,-6.99,7.23,6.75,-6.99,6.55,99.00,99.00,99.00,99.00,0.49,-0.53,-6.94,-0.49,99.00,6.46,-0.53,99.00,99.00,-7.86,-0.34,99.00,-6.94,99.00,...,0.49,-0.24,99.00,99.00,-3.11,-6.65,99.00,-0.58,6.31,99.00,99.00,-7.86,99.00,99.00,